In [2]:
import numpy as np
import os

In [3]:
PATH = './linearRegression_data'

In [4]:
TRAIN_LABEL_FILENAME = 'orientations_train.npy'
TRAIN_LABEL_FULL_PATH = os.path.join(PATH, TRAIN_LABEL_FILENAME)

y_train = np.load(TRAIN_LABEL_FULL_PATH)

In [5]:
TRAIN_SET_FOLDER_NAME = '3dshapes_train'
TRAIN_SET_FULL_PATH = os.path.join(PATH, TRAIN_SET_FOLDER_NAME)

TRAIN_SIZE = len(os.listdir(TRAIN_SET_FULL_PATH))
X_train = np.zeros((TRAIN_SIZE, 4096))

In [6]:
import cv2

In [7]:
for i in range(TRAIN_SIZE):
    img = cv2.imread(os.path.join(TRAIN_SET_FULL_PATH, '{}.jpg'.format(i)))
    gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    flat_img = gray_scale.flatten()
    X_train[i] = flat_img

In [8]:
print(X_train.shape)

(10000, 4096)
